# Setting enviroment

In [4]:
%load_ext kedro.ipython

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython


# Load data

In [98]:
df_customers = catalog.load("olist_customers_dataset")
df_geo = (
    catalog.load("olist_geolocation_dataset")
    .assign(
        geolocation_city = lambda df: (
            df
            .groupby('geolocation_zip_code_prefix')
            .geolocation_city.transform('first')
            .str.normalize('NFKD')
            .str.encode('ascii', errors='ignore')
            .str.decode('utf-8')
            .str.strip()
        ),
        geolocation_state = lambda df: (
            df
            .groupby('geolocation_zip_code_prefix')
            .geolocation_state.transform('first')
        )
    )
    .filter(['geolocation_zip_code_prefix', 'geolocation_city','geolocation_state'])
    .drop_duplicates()
)
df_items = catalog.load("olist_order_items_dataset")
df_pay = catalog.load("olist_order_payments_dataset")
df_rev = catalog.load("olist_order_reviews_dataset")
df_orders = catalog.load("olist_orders_dataset")
df_prd = catalog.load("olist_products_dataset")
df_sel = catalog.load("olist_sellers_dataset")


[12/23/24 14:29:22] INFO     Loading data from olist_customers_dataset (CSVDataset)...          ]8;id=143228;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=638161;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from olist_geolocation_dataset (CSVDataset)...        ]8;id=569695;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=480476;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

[12/23/24 14:29:24] INFO     Loading data from olist_order_items_dataset (CSVDataset)...        ]8;id=767156;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=913527;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

[12/23/24 14:29:25] INFO     Loading data from olist_order_payments_dataset (CSVDataset)...     ]8;id=124688;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=567015;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from olist_order_reviews_dataset (CSVDataset)...      ]8;id=947554;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=71309;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from olist_orders_dataset (CSVDataset)...             ]8;id=540079;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=615345;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from olist_products_dataset (CSVDataset)...           ]8;id=996720;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=998702;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from olist_sellers_dataset (CSVDataset)...            ]8;id=348261;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=260430;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

# Pagamentos de pedidos

In [185]:
print(
    "pode haver mais de um tipo de pagamento por pedido. "
    "\nno geral, esses pedidos com mais de um tipo de pagamento é por que o cliente "
    "\ncoloca cupons no momento da compra"
)
display(df_pay.groupby('order_id').payment_type.nunique().sort_values())
print("por exemplo")
display(df_pay.query('order_id=="62b2d948f0d2de4fe48b034bbdedf94c"'))

pode haver mais de um tipo de pagamento por pedido. 
no geral, esses pedidos com mais de um tipo de pagamento é por que o cliente 
coloca cupons no momento da compra



order_id
00010242fe8c5a6d1ba2dd792cb16214    1
aa3d4b2c0fb6da178364768f0d3008f2    1
aa3d230ecbebaa35cea8fe4c24739dbc    1
aa3bb8a5c7740d386e6318f44046ac14    1
aa3babada56030c2ad79976c60d51e79    1
                                   ..
6a868efa34dccad934e939877994dafe    2
2dcb16f827ccfc7bff5be2a6deb28bce    2
0c3e2dc4a3eeb282bd4df0b620dcff6d    2
257006b4652ed6214a889913fec6a5ca    2
62b2d948f0d2de4fe48b034bbdedf94c    2
Name: payment_type, Length: 99440, dtype: int64

por exemplo


,order_id,payment_sequential,payment_type,payment_installments,payment_value
17668,62b2d948f0d2de4fe48b034bbdedf94c,2,voucher,1,250.00
59535,62b2d948f0d2de4fe48b034bbdedf94c,1,credit_card,8,87.52


In [188]:

print(
    "mesmo tirando os cupons ainda pode haver mais de um tipo de pagamento por pedido"
)
display(df_pay.query('payment_type!="voucher"').groupby('order_id').payment_type.nunique().sort_values())
print("mas isso só acontece em um pedido:")
display(df_pay.query('order_id=="a079628ac8002126e75f86b0f87332e4"').sort_values('payment_sequential'))

mesmo tirando os cupons ainda pode haver mais de um tipo de pagamento por pedido



order_id
00010242fe8c5a6d1ba2dd792cb16214    1
ab5628545011511727a47e013585705d    1
ab53b19e9f59776c6556ebf49e85a52c    1
ab537f9c58a21747d30d02a43d2763a7    1
ab532e8d49a6cd8848ec641843b39b93    1
                                   ..
557de3912df820904b9204e0d1879256    1
557d6978e67b46040feebb5bf50e62d2    1
557cc2675910c4a6c9b54ad276f25097    1
557c1116cc609730b617567dec5790b3    1
a079628ac8002126e75f86b0f87332e4    2
Name: payment_type, Length: 97819, dtype: int64

mas isso só acontece em um pedido:


,order_id,payment_sequential,payment_type,payment_installments,payment_value
36471,a079628ac8002126e75f86b0f87332e4,2,debit_card,1,50.00
10258,a079628ac8002126e75f86b0f87332e4,3,credit_card,3,102.82


# Review de pedidos

In [ ]:
print("pode haver mais de um review para cada pedido")
display(df_rev.order_id.value_counts())
print("por exemplo")
df_rev.query('order_id=="8e17072ec97ce29f0e1f111e598b0c85"')


order_id
c88b1d1b157a9999ce368f218a407141    3
8e17072ec97ce29f0e1f111e598b0c85    3
df56136b8031ecd28e200bb18e6ddb2e    3
03c939fd7fd3b38f8485a0f95798f1f6    3
5cb890a68b91b6158d69257e4e2bc359    2
                                   ..
5b4e9a12d219f34f5c2de9f8d620b19d    1
a6da096d974acc000962856d7386448a    1
75e0647c26de647eca3421e9cc66c9da    1
bad0467c52f23cdc71e9fa139d4a8afd    1
90531360ecb1eec2a1fbb265a0db0508    1
Name: count, Length: 98673, dtype: int64

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
44694,67c2557eb0bd72e3ece1e03477c9dff5,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,Entregou o produto errado.,2018-04-07 00:00:00,2018-04-08 22:48:27
64510,2d6ac45f859465b5c185274a1c929637,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,Comprei 3 unidades do produto vieram 2 unidade...,2018-04-07 00:00:00,2018-04-07 21:13:05
92300,6e4c4086d9611ae4cc0cc65a262751fe,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,"Embora tenha entregue dentro do prazo, não env...",2018-04-14 00:00:00,2018-04-16 11:37:31


In [164]:
(
    df_orders
    .merge(df_pay,
           on=['order_id'],
           validate='1:m',
           how='left',
           indicator=True)
    # ._merge.value_counts()
    .query('_merge!="both"')
)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_sequential,payment_type,payment_installments,payment_value,_merge
32133,bfbd0f9bdef84302105ad712db648a6c,86dc2ffce2dfff336de2f386a786e574,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-07 17:11:53,2016-11-09 07:47:38,2016-10-04 00:00:00,NaN,NaN,NaN,NaN,left_only


# Carrinhos de compras

In [180]:
print('pode haver mais de um vendedor em cada pedido')
display(
    df_items
    .groupby('order_id').seller_id.nunique()
    .sort_values()
)
print("por exemplo: ")
display(df_items.query('order_id=="1c11d0f4353b31ac3417fbfa5f0f2a8a" '))

print("Mas para cada produto em um pedido há apenas um único vendedor :")
display(
    df_items
    .groupby(['order_id','product_id']).seller_id.nunique()
    .sort_values()
)

pode haver mais de um vendedor em cada pedido



order_id
00010242fe8c5a6d1ba2dd792cb16214    1
aaaf314a8cf0d0da71e52c6cd4184cbd    1
aaaea350ff8a957595f3c631d6b63d1b    1
aaae80f5b6239bd9e1b22e9aa542c3e8    1
aaabf43feb9498d9de4588eb73231c25    1
                                   ..
1d23106803c48c391366ff224513fb7f    4
8c2b13adf3f377c8f2b06b04321b0925    4
91be51c856a90d7efe86cf9d082d6ae3    4
cf5c8d9f52807cb2d2f0a0ff54c478da    5
1c11d0f4353b31ac3417fbfa5f0f2a8a    5
Name: seller_id, Length: 98666, dtype: int64

por exemplo: 


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
12378,1c11d0f4353b31ac3417fbfa5f0f2a8a,1,d64738c7b2e2d1778a51e835d8f02568,609e1a9a6c2539919b8205cf7c4e6ff0,2018-02-02 11:31:03,43.90,12.77
12379,1c11d0f4353b31ac3417fbfa5f0f2a8a,2,a3a10562c9d134b92f04b8cce298d037,822166ed1e47908f7cfb49946d03c726,2018-02-02 11:31:03,22.49,25.54
12380,1c11d0f4353b31ac3417fbfa5f0f2a8a,3,d80fce9f775927c7d602c2de926759f1,747c1cdcd1737dcfbd1547f6eb6f2cf3,2018-02-02 11:31:03,88.00,12.77
12381,1c11d0f4353b31ac3417fbfa5f0f2a8a,4,ddbf57e4559f03c003fc3708d4860c5e,1835b56ce799e6a4dc4eddc053f04066,2018-02-02 11:31:03,29.99,12.77
12382,1c11d0f4353b31ac3417fbfa5f0f2a8a,5,a3a10562c9d134b92f04b8cce298d037,822166ed1e47908f7cfb49946d03c726,2018-02-02 11:31:03,22.49,25.54
12383,1c11d0f4353b31ac3417fbfa5f0f2a8a,6,d3a386ad58b99f0ddcc216ed9efb03c3,609e1a9a6c2539919b8205cf7c4e6ff0,2018-02-02 11:31:03,43.90,12.77
12384,1c11d0f4353b31ac3417fbfa5f0f2a8a,7,61b6e5d2e3ee58d2b341b8ef1a652b26,c7fdb77fdbff3c41981bc52f787e959e,2018-02-02 11:31:03,56.00,12.77


Mas para cada produto em um pedido há apenas um único vendedor :



order_id                          product_id                      
00010242fe8c5a6d1ba2dd792cb16214  4244733e06e7ecb4970a6e2683c13e61    1
ab5048879a15f9f713725c37c5eb43ef  adae758556f8338f07f3064736416064    1
ab500cb1d60a0e2ae7ce02ef4e8236d9  8473872cbf0ba205b447e8651bd6e03e    1
ab4fc7cc026a9ea1a616584f9cfe4b9a  bb6b6e173472747c5b94c0b72c4fe56c    1
ab4f963cf3f826eda5b1b000a7af4de8  1279fba7025d91dadde108559a8d0485    1
                                                                     ..
556689f3e30f4662d3ec219bdc556b93  915ff35b0555b98fe64bc60674da4c43    1
5564c14d026bf6c92241f070bb98fb5b  b84c4c7f1845f068f40e350865b803fa    1
5561adcb0fd46da4cad3048fa4e7fc00  4d071a22b9cc548b64a75075a4c63151    1
556bb5c8ecb64be5ee1ee039d4376432  f5d5fa2bc95883494c61ae05a351348a    1
fffe41c64501cc87c801fd61db3f6244  350688d9dc1e75ff97be326363655e01    1
Name: seller_id, Length: 102425, dtype: int64

In [ ]:

df_order_prd_seller = (
    df_items
    .merge(df_prd,
           on=['product_id'],
           validate='m:1',
           how='left',
           indicator=True)
    .rename(columns={'_merge':'_merge_itm_prd'})
)

assert all(df_order_prd_seller._merge_itm_prd=="both")
# o dataset resultante deve ter o mesmo tamnho 
assert len(df_order_prd_seller) == len(df_items)

# Clientes

In [202]:
print("só existem 1 customer_id por linha na base")
display(df_customers.customer_id.value_counts())
print("Mas podem ter vários customer_unique_id para cada linha")
display(df_customers.customer_unique_id.value_counts())
print("por exemplo:")
display(df_customers.query("customer_unique_id=='8d50f5eadf50201ccdcedfb9e2ac8455'"))

só existem 1 customer_id por linha na base



customer_id
06b8999e2fba1a1fbc88172c00ba8bc7    1
c023f30c1147aeb0358474f3b1dbc707    1
b5cbf43f42281920a175fc99650c91d6    1
19f4e983f8f05b7de16670f8a0cf0ac7    1
229ac14c6ee6d6a606264ebcc861beb7    1
                                   ..
31665bd5bc542687ab54c9fd9ed9917b    1
37fb9cce079a4887c96b59bae2011d05    1
1d815f3ad9b2017b6da2c4f1cbb1180b    1
6b05c8f3a25a71005d5eb9b51766c333    1
274fa6071e5e17fe303b9748641082c8    1
Name: count, Length: 99441, dtype: int64

Mas podem ter vários customer_unique_id para cada linha



customer_unique_id
8d50f5eadf50201ccdcedfb9e2ac8455    17
3e43e6105506432c953e165fb2acf44c     9
1b6c7548a2a1f9037c1fd3ddfed95f33     7
ca77025e7201e3b30c44b472ff346268     7
6469f99c1f9dfae7733b25662e7f1782     7
                                    ..
081f07439678af2da7755a0aa572154d     1
9c57789a6a587f4efcfb01e36c106014     1
9e4c21635f640562de1a6976374fba03     1
06c0ea6ee892364d1608ee47aa9f56a0     1
84732c5050c01db9b23e19ba39899398     1
Name: count, Length: 96096, dtype: int64

por exemplo:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
14186,1bd3585471932167ab72a84955ebefea,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
15321,a8fabc805e9a10a3c93ae5bff642b86b,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
16654,897b7f72042714efaa64ac306ba0cafc,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
36122,b2b13de0770e06de50080fea77c459e6,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
38073,42dbc1ad9d560637c9c4c1533746f86d,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
40141,dfb941d6f7b02f57a44c3b7c3fefb44b,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
48614,65f9db9dd07a4e79b625effa4c868fcb,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
52574,1c62b48fb34ee043310dcb233caabd2e,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
58707,a682769c4bc10fc6ef2101337a6c83c9,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
67996,6289b75219d757a56c0cce8d9e427900,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP


In [222]:
print("as informações de um customer_unique_id mudam em diferentes linhas? ")
df_aux = (
    df_customers
    .groupby('customer_unique_id')
    .nunique()
    .drop(columns=['customer_id'])
    .sum(axis=1)
    .sort_values()
)
display(
    df_aux
)
print(f"Sim, {(df_aux>3).mean():2.2%} mudam. Por exemplo:")
display(
    df_customers.query("customer_unique_id=='d44ccec15f5f86d14d6a2cfa67da1975'")
)

as informações de um customer_unique_id mudam em diferentes linhas? 



customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    3
aa53470c30cfd5c372dd45a57203e529    3
aa514cd67b53c21de47c7dc5972555e7    3
aa5014f5a39d8869707206c78f15ba07    3
aa4fc6bd5cf9d541b5a808cdada1301c    3
                                   ..
ee4a5dab94b427525ea24bc30d4442fb    6
8c21dd8c37144807c601f99f2a209dfb    6
2c45ab66a3dae52960147e76a35740ff    6
b9badb100ff8ecc16a403111209e3a06    7
d44ccec15f5f86d14d6a2cfa67da1975    9
Length: 96096, dtype: int64

Sim, 0.26% mudam. Por exemplo:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
45851,844a7e5d2737f2f6667a5242486cbcb1,d44ccec15f5f86d14d6a2cfa67da1975,3533,sao paulo,SP
57951,105e31e3b36d904c70058578126c4d12,d44ccec15f5f86d14d6a2cfa67da1975,62800,aracati,CE
83347,488dd91f70d8404b2aa677cdc8a0de68,d44ccec15f5f86d14d6a2cfa67da1975,88371,navegantes,SC


In [ ]:
assert all(df_customers.customer_id != df_customers.customer_unique_id)

In [ ]:
# quantos pedidos por cada clientes:
n_customer_with_n_orders_gt_1 = (
    df_customers
    .merge(
        df_orders,
        on='customer_id',
        validate='1:m',
        how='left',
    )
    .groupby('customer_unique_id')
    .order_id.nunique()
    .sort_values()
     > 1
).sum()
n_customers = df_customers.customer_unique_id.nunique()
print(f'Dos {n_customers} apenas {n_customer_with_n_orders_gt_1}({n_customer_with_n_orders_gt_1/n_customers:2.2%}) tem mais de um pedido.')
print('Isso inviabiliza problemas a nível de clientes, pois há pouco histórico de compras do clintes. Por exemplo, churn de cliente')
print("Isto é, não existem clientes com histórico muito longo na plataforma")

Dos 96096 apenas 2997(3.12%) tem mais de um pedido.


# Vendedores

In [249]:
print(f"Existem {df_sel.seller_id.nunique()} vendedores na base.")

Existem 3095 vendedores na base.


In [250]:
df_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [260]:
n_sellers = df_sel.seller_id.nunique()
s = (
    df_sel
    .merge(df_items[['seller_id','order_id']], on=['seller_id'], how='left')
    .groupby('seller_id')
    .order_id.nunique()
)
n_sellers_with_gt_1 = len(s[s>1])
display(s.describe())
print(f"Dos {n_sellers} vendedores {n_sellers_with_gt_1}({n_sellers_with_gt_1/n_sellers:2.2%}) tem mais de uma venda")
print("Parece ser possível fazer um problema de machine learning a nível de vendedores")
print("Um exemplo é o risco do vendedor ficar sem vender nos próximos x meses")
print("Outro exemplo é o volume de vendas (quantidade ou monetário)"
      " de um vendedor nos próximos x meses")


count    3095.000000
mean       32.313409
std       105.139763
min         1.000000
25%         2.000000
50%         6.000000
75%        21.500000
max      1854.000000
Name: order_id, dtype: float64

Dos 3095 vendedores 2524(81.55%) tem mais de uma venda
Parece ser possível fazer um problema de machine learning a nível de vendedores
Um exemplo é o risco do vendedor ficar sem vender nos próximos x meses
Outro exemplo é o volume de vendas (quantidade ou monetário) de um vendedor nos próximos x meses


In [262]:
(
    df_sel
    .merge(df_items[['seller_id','order_id']],
           on=['seller_id'],
           how='outer',
           indicator=True)
    ._merge.value_counts(normalize=True)
)


_merge
both          1.0
left_only     0.0
right_only    0.0
Name: proportion, dtype: float64

# Pedidos

In [245]:
df_orders.order_purchase_timestamp.min(),df_orders.order_purchase_timestamp.max()

('2016-09-04 21:15:19', '2018-10-17 17:30:18')

In [264]:
df_orders.order_id.value_counts()


order_id
e481f51cbdc54678b7cc49136f2d6af7    1
f01059d0d674e1282df4e8fbbe015aa2    1
fbc17f0f2a2125054d5ac5c22d2d5120    1
9373150545066777b1cd2bc20e93cf8e    1
917399e96f92268dfa2c0351b1b75fba    1
                                   ..
6b8986012d61963295ffa3ea869aff86    1
be879f757debd3b384b540daa6ddd97e    1
ac3ce7eda1246f39509a505242fcc169    1
e801a93b3904ca9e8350f176a037047b    1
66dea50a8b16d9b4dee7af250b4be1a5    1
Name: count, Length: 99441, dtype: int64

In [268]:
df_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [270]:
df_items[['order_id','product_id','order_item_id']].value_counts()


order_id                          product_id                        order_item_id
00010242fe8c5a6d1ba2dd792cb16214  4244733e06e7ecb4970a6e2683c13e61  1                1
ab114f47094abcacaf5888844db683ff  1efd84ba20d23186f576a37baf64ae9d  1                1
ab14fdcfbe524636d65ee38360e22ce8  9571759451b1d780ee7c15012ea109d4  4                1
                                                                    3                1
                                                                    2                1
                                                                                    ..
553a1f1b74cf15fe6cb4a01a627ab41f  e8316a4667e5870c85e906b1f062bde1  1                1
553a0f2feba6fd288aaea2986ca2f94a  d9bdf643d95cb89844c0da1a0df1d16e  1                1
5539bd029cf95d97ba8f51f6f323c839  3bab56924ec762e9340478909312d5c7  1                1
553901a853048dcd33ec8de19f90c5d0  ac034106148bc5d510565d4b640f74c2  1                1
fffe41c64501cc87c801fd61db3f6244  350688d9dc1e7

In [ ]:
# faz sentido considerar apenas os pedidos entregues
# já que os demais tem poucos casos
(
    df_orders
    .merge(df_items[['seller_id','order_id']].drop_duplicates(),
           on=['order_id'],
           how='left',
           indicator=True)
    .rename(columns={'_merge':'_merge_order_item'})
    [["_merge_order_item","order_status"]].value_counts(normalize=True)
    .sort_index()
)


_merge_order_item  order_status
left_only          canceled        0.001627
                   created         0.000050
                   invoiced        0.000020
                   shipped         0.000010
                   unavailable     0.005983
both               approved        0.000020
                   canceled        0.004574
                   delivered       0.970571
                   invoiced        0.003096
                   processing      0.002987
                   shipped         0.010994
                   unavailable     0.000069
Name: proportion, dtype: float64

In [288]:
df_orders.columns


Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [ ]:
(
    df_orders
    .query("order_status=='delivered'")
    .merge(df_items[['seller_id','order_id']].drop_duplicates(),
           on=['order_id'],
           how='left')
)

2970